# Environment set up

In [1]:
# the endpoint generated by the cloudformation stack for neptune
endpoint = 'to be copied from CF'

In [2]:
import neptune_ml_utils as neptune_ml
neptune_ml.check_ml_enabled()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
This Neptune cluster is configured to use Neptune ML


In [3]:
from gremlin_python import statics
from gremlin_python.structure.graph import Graph
from gremlin_python.process.graph_traversal import __
from gremlin_python.process.strategies import *
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.driver.aiohttp.transport import AiohttpTransport
from gremlin_python.process.traversal import *
from gremlin_python.process.graph_traversal import id_
import os

In [4]:
s3_bucket_uri="s3://telcograph"

# Prepare the PoC for the Transductive mode




- drop some edges : Drop some links between some existing users and Cells to predict them with the transductive mode training.
- training is already done and the model_transductive is available in the Endpoint
- the transductive training take into account all users and cells properties as well as the user_live_cell properties. 
- for user_0 and cell_62000 we dropped ALL edges
- for user_1500 and cell_56500 we dropped ALL edges  
- for user_4570 three edges user_live_cell_1678350, user_live_cell_734598, user_live_cell_487137

### Select users to drop their user_live_cell edges

In [58]:
%%gremlin
g.V()
.hasId("user_0")
.outE()
.hasLabel("user_live_cell")
.inV()
.valueMap(true, "name")
.groupCount()
.unfold()
.order()
.by(values, desc)

In [34]:
%%gremlin
g.V('user_0').outE()

In [69]:
%%gremlin
g.V('user_0').bothE().where(otherV().hasId('cell_62000'))

In [7]:
#%%gremlin
#g.V('user_0').bothE().where(otherV().hasId('cell_62000')).drop()

In [83]:
%%gremlin
g.V()
.hasId("user_1500")
.outE()
.hasLabel("user_live_cell")
.inV()
.valueMap(true, "name")
.groupCount()
.unfold()
.order()
.by(values, desc)

In [84]:
%%gremlin
g.V('user_1500').bothE().where(otherV().hasId('cell_56500'))

In [73]:
%%gremlin
g.V()
.hasId("user_1500")
.outE()
.hasLabel("user_live_cell")
.inV()
.valueMap(true, "name")
.groupCount()
.unfold()
.order()
.by(values, desc)

In [74]:
%%gremlin
g.V('user_4570').bothE().where(otherV().hasId('cell_10570'))

In [75]:
%%gremlin
g.V('user_4570').outE('user_live_cell').hasId('user_live_cell_734598').drop()

In [44]:
#%%gremlin
#g.V('user_4570').outE('user_live_cell').hasId('user_live_cell_1678350').drop()

In [45]:
%%gremlin
g.V('user_4570').bothE().where(otherV().hasId('cell_10570'))

# Launch the export
## Frame the link prediction task in the export_param

In [35]:
export_params={ 
"command": "export-pg", 
"params": { "endpoint": neptune_ml.get_host(),
            "profile": "neptune_ml",
            "useIamAuth": neptune_ml.get_iam(),
            "cloneCluster": False,
            "nodeLabels": ["user", "cell"],
            "edgeLabels": ["user_live_cell"]
            }, 
"outputS3Path": f'{s3_bucket_uri}/neptune-export',
"additionalParams": {
        "neptune_ml": {
          "version": "v2.0",
          "targets": [
            {
                "edge": ["user", "user_live_cell", "cell"],
                "type" : "link_prediction",
                "split_rate": [0.8, 0.1, 0.1]
            }
         ]
        }
      },
"jobSize": "xlarge"}
export_params

{'command': 'export-pg',
 'params': {'endpoint': 'cluster.cluster-c2wgvyuhzgmx.eu-west-1.neptune.amazonaws.com',
  'profile': 'neptune_ml',
  'useIamAuth': False,
  'cloneCluster': False,
  'nodeLabels': ['user', 'cell'],
  'edgeLabels': ['user_live_cell']},
 'outputS3Path': 's3://telcograph/neptune-export',
 'additionalParams': {'neptune_ml': {'version': 'v2.0',
   'targets': [{'edge': ['user', 'user_live_cell', 'cell'],
     'type': 'link_prediction',
     'split_rate': [0.8, 0.1, 0.1]}]}},
 'jobSize': 'xlarge'}

In [147]:
%%neptune_ml export start --export-url {neptune_ml.get_export_service_host()} --export-iam --wait --store-to export_results
${export_params}

Output()

In [36]:
# check the status
%neptune_ml export status --export-url {neptune_ml.get_export_service_host()} --export-iam --job-id 4660f4dd-bcdf-4abe-949a-692ddd54f593

Output()

# Data processing/Preparation of graph data for Training

NB. Precise in export_param the S3 with the latest export to consider 

In [37]:
# The training_job_name can be set to a unique value below, otherwise one will be auto generated
training_job_name=neptune_ml.get_training_job_name('link-prediction')

processing_params = f"""
--config-file-name training-data-configuration.json
--job-id {training_job_name} 
--instance-type ml.r5.16xlarge
--s3-input-uri s3://telcograph/neptune-export/20230518_072439
--s3-processed-uri {str(s3_bucket_uri)}/preloading """

In [22]:
%neptune_ml dataprocessing start --wait --store-to processing_results {processing_params}

Output()

# Training 

<div style="background-color:#eeeeee; padding:20px; text-align:left; border-radius:10px; margin-top:10px; margin-bottom:10px; "><b>Information</b>: Link prediction is a more computationally complex model than classification or regression </div>

## Transductive Training steps 

In [1]:
#check the Ids for training and processing jobs
training_job_name = "my-train-job-ept-hypmodv9"
training_job_name

'my-train-job-ept-hypmodv9'

In [7]:
!curl -s "https://cluster.cluster-c2wgvyuhzgmx.eu-west-1.neptune.amazonaws.com:8182/ml/dataprocessing/"

{"ids":["link-prediction-1690116689","link-prediction-1687117627","link-prediction-1684928671"]}

In [8]:
!curl -s "https://cluster.cluster-c2wgvyuhzgmx.eu-west-1.neptune.amazonaws.com:8182/ml/dataprocessing/link-prediction-1684928671"

{"processingJob":{"name":"link-pre-preloading-2023-05-24-11-44-390000","arn":"arn:aws:sagemaker:eu-west-1:134846823987:processing-job/link-pre-preloading-2023-05-24-11-44-390000","status":"Completed","outputLocation":"s3://telcograph/preloading/link-pre-preloading-2023-05-24-11-44-390000/preloading-output"},"id":"link-prediction-1684928671","status":"Completed"}

In [39]:
training_params=f"""
--job-id {training_job_name}
--data-processing-id {training_job_name}
--instance-type ml.g4dn.16xlarge
--s3-output-uri {str(s3_bucket_uri)}/training
--max-hpo-number 1
--max-hpo-parallel 3 """

In [ ]:
training_params

In [27]:
%neptune_ml training start --wait --store-to training_results {training_params}

Output()

In [41]:
training_params

'\n--job-id my-train-job-ept-hypmodv9\n--data-processing-id my-train-job-ept-hypmodv9\n--instance-type ml.g4dn.16xlarge\n--s3-output-uri s3://telcograph/training\n--max-hpo-number 1\n--max-hpo-parallel 3 '

In [43]:
%neptune_ml training status --job-id my-train-job-ept-hypmodv9

Output()

# Inference 

## Endpoint creation 

In [ ]:
endpoint_params=f"""
--id {training_job_name}
--model-training-job-id {training_job_name}"""
endpoint_params

In [ ]:
%neptune_ml endpoint create --wait --store-to endpoint_results {endpoint_params}

In [ ]:
endpoint_transductive=endpoint_results['endpoint']['name']

In [5]:
endpoint_transductive ='my-train-2023-08-29-10-12-7520000-endpoint'

### reminder

- user_0 and cell_62000 / dropped ALL edges 

- user_1500 and cell_56500 / dropped ALL edges 

- user_4570 three edges of cell_10570

In [6]:
import boto3
import json
import time

def invoke_sagemaker_endpoint(endpoint_name, vertex_id, edge_type, head_node_type=None, tail_node_type=None, 
    exclude_flag='null', topk=10, response_format='application/json', region='eu-west-1'):
    if tail_node_type:
        input_data = {
            "version": "v1",
            "mode": "predict_tail",
            "data": {
                "globalParameters": {
                    'topk': topk,
                    'edgeType': edge_type,
                    'tailNodeType': tail_node_type,
                    'excludeFlag': exclude_flag
                },
                "edges": [
                    {
                        "headNodeId": vertex_id,
                    }
                ]
            }
        }
    else:
        input_data = {
            "version": "v1",
            "mode": "predict_head",
            "data": {
                "globalParameters": {
                    'topk': topk,
                    'edgeType': edge_type,
                    'headNodeType': head_node_type,
                    'excludeFlag': exclude_flag
                },
                "edges": [
                    {
                        "tailNodeId": vertex_id,
                    }
                ]
            }
        }
    sagemaker_runtime = boto3.client('sagemaker-runtime', region_name=region)
    payload = json.dumps(input_data)
    tic = time.time()
    response = sagemaker_runtime.invoke_endpoint(EndpointName=endpoint_name,
                                                 ContentType='application/json',
                                                 Accept=response_format,
                                                 Body=payload)
    duration = time.time() - tic
    print()
    output = json.loads(response['Body'].read().decode())["output"]
    for result in output["edges"][0]["mlResults"]:
        print(result['inferredValue'])
    print("Inference took {} milliseconds".format(duration*1000))

<img src="attachment:2a0d7696-5e11-42af-a56f-f7acc5d63572.png" alt="image.png" width="1000"/>

## DEMO shows: how GNN predicts to which cells, given user will be connected or Predicting which users are going to connect to a given cell 

<div style="background-color:#eeeeee; padding:20px; text-align:left; border-radius:10px; margin-top:10px; margin-bottom:10px; "><b>Experimentation1</b>: GNN is going to predict that user_0 is connected to cell_62000</div>

In [7]:
%%gremlin
g.with("Neptune#ml.endpoint","${endpoint_transductive}")
.with("Neptune#ml.limit", 5)
.V("cell_62000")
.in("user_live_cell").with("Neptune#ml.prediction").hasLabel("user")

Keyboard interrupt detected.


<div style="background-color:#eeeeee; padding:20px; text-align:left; border-radius:10px; margin-top:10px; margin-bottom:10px; "><b>Experimentation2</b>: GNN is going to predict that user_1500 is connected to cell_56500</div>

In [30]:
%%gremlin
g.with("Neptune#ml.endpoint","${endpoint_transductive}")
.with("Neptune#ml.limit",10)
.V("cell_56500")
.in("user_live_cell").with("Neptune#ml.prediction").hasLabel("user")

<div style="background-color:#eeeeee; padding:20px; text-align:left; border-radius:10px; margin-top:10px; margin-bottom:10px; "><b>Experimentation3</b>: GNN is going to predict that user_1500 is connected to cell_63500</div>

In [28]:
%%gremlin
g.with("Neptune#ml.endpoint","${endpoint_transductive}")
.with("Neptune#ml.limit",5)
.V("user_1500")
.out("user_live_cell").with("Neptune#ml.prediction").with("Neptune#ml.filterExistingEdges").hasLabel("cell")

# end to end architecture for multi- scenario 

<img src="attachment:f8b596d3-26cd-4127-aebb-132900d153db.png" alt="image.png" width="1000"/>

# Note on GNN evaluation 

- HITS@10 provides a measure of how often the model suggests the correct item within the top 10 recommendation

- While MR gives an overall indication of how close the correct item is to the top of the list on average.

- Sagemaker evaluate the model on the train and test set 

- Results on test set 

    * "HITS at top 1 (HITS@1)": 0.4010819758391616,
    * "HITS at top 10 (HITS@10)": 0.9438622262173598,
    * "HITS at top 3 (HITS@3)": 0.6301810418539388,
    * "mean rank (MR)": 3.719502285632852,
